In [1]:
import pandas as pd
import sqlite3 as sql


In [2]:
df_table1 = pd.read_csv('users.csv')
df_table2 = pd.read_csv('menu.csv')
df_table3 = pd.read_csv('restaurants.csv')
df_table4 = pd.read_csv('orders.csv')
df_table5 = pd.read_csv('order_details.csv')
df_table6 = pd.read_csv('food.csv')
df_table7 = pd.read_csv('delivery_partner.csv')

In [3]:
conn = sql.connect(':memory:')

In [4]:
df_table1.to_sql('users', conn, index=False)
df_table2.to_sql('menu', conn, index=False)
df_table3.to_sql('restaurants', conn, index=False)
df_table4.to_sql('orders', conn, index=False)
df_table5.to_sql('order_details', conn, index=False)
df_table6.to_sql('food', conn, index=False)
df_table7.to_sql('delivery_partner', conn, index=False)

5

In [5]:
query ="select * from restaurants"

#we are just carrying out the list of restaurants from restauarant table

In [6]:
result = pd.read_sql_query(query, conn)

In [7]:
print(result)

   r_id      r_name       cuisine
0     1     dominos       Italian
1     2         kfc      American
2     3        box8  North Indian
3     4  Dosa Plaza  South Indian
4     5  China Town       Chinese


In [8]:
query1 ="""
select price
FROM menu m """


#in this query we are listing out the price of each menu

In [9]:
result2 = pd.read_sql_query(query1, conn)

print(result2)

    price
0     450
1     400
2     100
3     115
4     230
5     300
6      80
7     160
8     140
9     230
10    180
11    120
12    250
13    220
14    180


In [10]:
query2 ="""
select name 
from users
where user_id NOT IN 
(SELECT user_id from orders)"""

#we have to find the user name where they havent ordered , so in this we select the user_id from orders means that user_id who placed the orders so from the users tables we get who havent ordered using NOT IN command

In [11]:
result3 = pd.read_sql_query(query2, conn)

print(result3)

      name
0  Anupama
1  Rishabh


In [12]:
query3 ="""
select r.r_name, count(*) as 'month'
from orders o
join restaurants r
on o.r_id=r.r_id
group by r.r_name
order by count(*) desc limit 1"""

#in this query we are finding the top restaurants in terms of number of orders for a given month

In [13]:
result4=pd.read_sql_query(query3, conn)
print(result4)

  r_name  month
0    kfc      8


In [14]:
query5="""
select r.r_name, SUM(amount) as 'revenue'
from orders o
join restaurants r
on r.r_id=o.r_id
group by r.r_name
having revenue > 500"""


#we have to finding out the total revenue of all months and we are carrying out the restauarant where revenue is greater than 500

In [15]:
result5=pd.read_sql_query(query5,conn)

In [16]:
print(result5)

       r_name  revenue
0  China Town     1450
1  Dosa Plaza     1480
2        box8      940
3     dominos     3050
4         kfc     3570


In [17]:
query6="""
select r.r_name, COUNT(*) AS 'LOYAL_CUSTOMERS'
FROM (select r_id,user_id,COUNT(*) AS 'visits'
      from orders
      group by r_id, user_id
      having visits>1) t
join restaurants r 
on r.r_id=t.r_id
group by t.r_id
order by LOYAL_CUSTOMERS
DESC LIMIT 1"""

#find restaurants with max repeated customers

In [18]:
result6 = pd.read_sql_query(query6,conn)

In [19]:
print(result6)

  r_name  LOYAL_CUSTOMERS
0    kfc                2


In [20]:
query7="""
select strftime('%m', o.date) AS 'month',SUM(amount)
from orders o
group by month """

#month over month revenue growth of swiggy#

In [21]:
result7=pd.read_sql_query(query7,conn)

In [22]:
print(result7)

  month  SUM(amount)
0    05         2425
1    06         3220
2    07         4845


In [38]:
query8="""
select month, ((revenue - prev)*100/prev) as 'monthyly growth' from ( 
    with Sales as 
   (
     select strftime('%m', o.date) AS 'month',SUM(amount) as 'revenue'
     from orders o
     group by month
     order by month
    )
   
   select month,revenue,LAG(revenue,1) over(order by revenue) as prev from Sales) t
"""

In [39]:
result8=pd.read_sql_query(query8,conn)

In [40]:
print(result8)

  month  monthyly growth
0    05              NaN
1    06             32.0
2    07             50.0


In [41]:
query9="""
select f.f_name, AVG(price) as "AVG PRICE"
FROM menu m
JOIN food f
on m.f_id = f.f_id
group by f.f_name"""

#in this we are carrying out the average price of each dish

In [42]:
result9=pd.read_sql_query(query9,conn)

In [43]:
print(result9)

              f_name   AVG PRICE
0    Chicken Popcorn  300.000000
1      Chicken Wings  230.000000
2    Choco Lava cake   98.333333
3        Masala Dosa  180.000000
4      Non-veg Pizza  450.000000
5          Rava Idli  120.000000
6          Rice Meal  213.333333
7          Roti meal  140.000000
8   Schezwan Noodles  220.000000
9     Veg Manchurian  180.000000
10         Veg Pizza  400.000000


#customer and his favourite food

In [51]:
query10="""
with temp as (
              select o.user_id,od.f_id,count(*) as 'frequency'
              from orders o
              join order_details od 
              on o.order_id =od.order_id
              group by o.user_id, od.f_id
              )
select u.name,f.f_name, t1.frequency
from temp t1
join users u
on u.user_id=t1.user_id
join food f 
on f.f_id =t1.f_id
where t1.frequency =(
                     select max(frequency)
                     from temp t2
                     where t2.user_id=t1.user_id)"""

#here we are finding out the favourite customer food where they have placed the same order for more number of time

In [52]:
result10=pd.read_sql_query(query10,conn)

In [53]:
print(result10)

       name            f_name  frequency
0    Nitish   Choco Lava cake          5
1  Khushboo   Choco Lava cake          3
2   Vartika     Chicken Wings          3
3     Ankit  Schezwan Noodles          3
4     Ankit    Veg Manchurian          3
5      Neha   Choco Lava cake          5


In [71]:
query11="""
select o.order_id, r.r_name,f.f_name
from orders o
join restaurants r
on r.r_id=o.r_id
join order_details od
on o.order_id=od.order_id
join food f
on f.f_id=od.f_id
where user_id = (select user_id 
                 from users u
                 where name LIKE 'Ankit' and (date > '2022-06-10' and date < '2022-07-10'))
"""


#in this query we have shown all orders with order details for a particular customer in a particular date range

In [72]:
result11=pd.read_sql_query(query11,conn)

In [73]:
print(result11)

   order_id      r_name            f_name
0      1018  Dosa Plaza  Schezwan Noodles
1      1018  Dosa Plaza    Veg Manchurian
2      1019  China Town  Schezwan Noodles
3      1019  China Town    Veg Manchurian
